In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math

In [2]:
#read Seattle sidewalk geometry
sidewalk = gpd.read_file('seattle-sidewalk/Sidewalks/Sidewalks.shp')
sidewalk = sidewalk.dropna(subset=['geometry'])

In [3]:
#read Project Sidewalk labels
labels = gpd.read_file('attributes.json')
labels

,attribute_id,label_type,street_edge_id,osm_street_id,neighborhood,avg_image_capture_date,avg_label_date,severity,is_temporary,agree_count,disagree_count,notsure_count,geometry
0,70839935,CurbRamp,14822,98239835,Harbor Island,2017-09-01,2019-04-26 21:46:21.100,3.0,False,1,0,0,POINT (-122.35248 47.58456)
1,70839936,CurbRamp,14822,98239835,Harbor Island,2014-06-01,2019-04-26 04:50:37.138,2.0,False,1,0,0,POINT (-122.35149 47.58473)
2,70839937,CurbRamp,14809,98239825,Harbor Island,2017-09-01,2019-04-22 00:58:33.914,2.0,False,1,0,0,POINT (-122.35236 47.57236)
3,70839938,CurbRamp,14826,98239858,Harbor Island,2017-09-01,2019-04-26 02:03:04.038,2.0,False,1,0,0,POINT (-122.35410 47.57295)
4,70839939,CurbRamp,14826,98239858,Harbor Island,2017-09-01,2019-04-26 11:36:21.207,2.0,False,1,0,1,POINT (-122.35539 47.57450)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
105443,77221398,Signal,14586,68307145,Matthews Beach,2022-08-01,2023-05-14 02:42:07.690,NaN,False,0,0,0,POINT (-122.29054 47.71189)
105444,77221399,Signal,1907,6368129,Matthews Beach,2022-09-01,2023-05-09 15:18:32.905,NaN,False,1,1,1,POINT (-122.29060 47.70853)
105445,77221400,Signal,1907,6368129,Matthews Beach,2022-09-01,2023-05-03 09:01:36.253,NaN,False,5,1,0,POINT (-122.29061 47.70829)
105446,77221401,Signal,16959,158794036,Matthews Beach,2022-09-01,2023-04-26 00:26:57.504,NaN,False,2,0,0,POINT (-122.29082 47.70483)


In [4]:
#select only curb ramps and no curb ramps
curbs = labels[labels['label_type'].isin(['CurbRamp', 'NoCurbRamp'])]
curbs.head()

,attribute_id,label_type,street_edge_id,osm_street_id,neighborhood,avg_image_capture_date,avg_label_date,severity,is_temporary,agree_count,disagree_count,notsure_count,geometry
0,70839935,CurbRamp,14822,98239835,Harbor Island,2017-09-01,2019-04-26 21:46:21.100,3.0,False,1,0,0,POINT (-122.35248 47.58456)
1,70839936,CurbRamp,14822,98239835,Harbor Island,2014-06-01,2019-04-26 04:50:37.138,2.0,False,1,0,0,POINT (-122.35149 47.58473)
2,70839937,CurbRamp,14809,98239825,Harbor Island,2017-09-01,2019-04-22 00:58:33.914,2.0,False,1,0,0,POINT (-122.35236 47.57236)
3,70839938,CurbRamp,14826,98239858,Harbor Island,2017-09-01,2019-04-26 02:03:04.038,2.0,False,1,0,0,POINT (-122.35410 47.57295)
4,70839939,CurbRamp,14826,98239858,Harbor Island,2017-09-01,2019-04-26 11:36:21.207,2.0,False,1,0,1,POINT (-122.35539 47.57450)


In [5]:
s = sidewalk.explode(ignore_index = True)

In [6]:
first_coord = s["geometry"].apply(lambda g: g.coords[0])
last_coord = s["geometry"].apply(lambda g: g.coords[-1])

In [7]:
a = pd.DataFrame(first_coord)
b = pd.DataFrame(last_coord)
df = pd.concat([a,b])

In [8]:
df

,geometry
0,"(-122.32181743343448, 47.605868653708114)"
1,"(-122.32137567122491, 47.6050411016493)"
2,"(-122.32115153522332, 47.60513478501772)"
3,"(-122.32087183301137, 47.604500123773846)"
4,"(-122.32084146538824, 47.60479544108653)"
...,...
46097,"(-122.36902311112672, 47.634704248485484)"
46098,"(-122.30338916373728, 47.72652791476223)"
46099,"(-122.2946810910384, 47.72466558425913)"
46100,"(-122.29409073065622, 47.55718043844571)"


In [9]:
points = df.apply(lambda row: Point(row), axis=1)
p_geo = gpd.GeoDataFrame(df, geometry=points)
p_geo.crs = {'init': 'epsg:4326'}
p_geo = p_geo.to_crs("EPSG:2285")

In [10]:
r =p_geo.buffer(60)
r1 = gpd.GeoDataFrame(geometry=gpd.GeoSeries(r))

In [31]:
curbs = curbs.to_crs("EPSG:2285")
curbs =curbs.drop(columns ={"avg_label_date","avg_image_capture_date"})

In [32]:
curbs_filtered = curbs.sjoin(r1, op="within", how="inner")

In [33]:
curbs_filtered

,attribute_id,label_type,street_edge_id,osm_street_id,neighborhood,severity,is_temporary,agree_count,disagree_count,notsure_count,geometry,index_right
0,70839935,CurbRamp,14822,98239835,Harbor Island,3.0,False,1,0,0,POINT (1265532.527 216929.573),33627
2,70839937,CurbRamp,14809,98239825,Harbor Island,2.0,False,1,0,0,POINT (1265474.804 212479.929),3804
17,70839952,CurbRamp,14809,98239825,Harbor Island,1.0,False,0,0,0,POINT (1265491.860 212485.159),3804
18,70839953,CurbRamp,14809,98239825,Harbor Island,3.0,False,1,0,0,POINT (1265457.555 212464.960),3804
43,70839978,CurbRamp,14809,98239825,Harbor Island,2.0,False,0,0,0,POINT (1265525.560 212474.751),3804
...,...,...,...,...,...,...,...,...,...,...,...,...
105413,77221368,NoCurbRamp,4960,6427434,Matthews Beach,3.0,False,0,0,0,POINT (1282874.556 258126.713),7360
105413,77221368,NoCurbRamp,4960,6427434,Matthews Beach,3.0,False,0,0,0,POINT (1282874.556 258126.713),7164
105413,77221368,NoCurbRamp,4960,6427434,Matthews Beach,3.0,False,0,0,0,POINT (1282874.556 258126.713),7165
105413,77221368,NoCurbRamp,4960,6427434,Matthews Beach,3.0,False,0,0,0,POINT (1282874.556 258126.713),13447


In [41]:
id_list= curbs_filtered['attribute_id'].unique().tolist()
curbs_false = curbs[~curbs['attribute_id'].isin(id_list)]

In [42]:
curbs_false

,attribute_id,label_type,street_edge_id,osm_street_id,neighborhood,severity,is_temporary,agree_count,disagree_count,notsure_count,geometry
1,70839936,CurbRamp,14822,98239835,Harbor Island,2.0,False,1,0,0,POINT (1265778.486 216987.347)
3,70839938,CurbRamp,14826,98239858,Harbor Island,2.0,False,1,0,0,POINT (1265049.760 212704.045)
4,70839939,CurbRamp,14826,98239858,Harbor Island,2.0,False,1,0,1,POINT (1264740.841 213275.200)
5,70839940,CurbRamp,14819,98239834,Harbor Island,2.0,False,1,0,0,POINT (1264749.214 213412.817)
8,70839943,CurbRamp,14826,98239858,Harbor Island,3.0,False,5,0,0,POINT (1264748.808 213106.642)
...,...,...,...,...,...,...,...,...,...,...,...
105391,77221346,NoCurbRamp,1551,6360029,Matthews Beach,2.0,False,1,0,0,POINT (1283220.091 259012.288)
105402,77221357,NoCurbRamp,10231,6514094,Matthews Beach,4.0,False,0,0,0,POINT (1283872.400 258429.394)
105407,77221362,NoCurbRamp,6257,6449067,Matthews Beach,5.0,False,0,0,0,POINT (1282128.503 259812.267)
105412,77221367,NoCurbRamp,5230,6431075,Matthews Beach,4.0,False,0,0,0,POINT (1283520.123 258082.530)


In [45]:
#put curb false agttribute id into a list
curbs_false_id = curbs_false['attribute_id'].unique().tolist()
labels = labels[~labels['attribute_id'].isin(curbs_false_id)]


In [46]:
labels

,attribute_id,label_type,street_edge_id,osm_street_id,neighborhood,avg_image_capture_date,avg_label_date,severity,is_temporary,agree_count,disagree_count,notsure_count,geometry
0,70839935,CurbRamp,14822,98239835,Harbor Island,2017-09-01,2019-04-26 21:46:21.100,3.0,False,1,0,0,POINT (-122.35248 47.58456)
2,70839937,CurbRamp,14809,98239825,Harbor Island,2017-09-01,2019-04-22 00:58:33.914,2.0,False,1,0,0,POINT (-122.35236 47.57236)
6,70839941,CurbRamp,14809,98239825,Harbor Island,2017-09-01,2019-04-26 11:45:42.880,3.0,False,2,0,0,POINT (-122.35294 47.57284)
7,70839942,CurbRamp,14809,98239825,Harbor Island,2016-01-16,2019-04-26 11:45:33.643,3.0,False,2,0,0,POINT (-122.35302 47.57291)
9,70839944,CurbRamp,14833,98239870,Harbor Island,2017-09-01,2019-04-21 23:21:35.297,2.0,False,4,0,0,POINT (-122.35578 47.57989)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
105443,77221398,Signal,14586,68307145,Matthews Beach,2022-08-01,2023-05-14 02:42:07.690,NaN,False,0,0,0,POINT (-122.29054 47.71189)
105444,77221399,Signal,1907,6368129,Matthews Beach,2022-09-01,2023-05-09 15:18:32.905,NaN,False,1,1,1,POINT (-122.29060 47.70853)
105445,77221400,Signal,1907,6368129,Matthews Beach,2022-09-01,2023-05-03 09:01:36.253,NaN,False,5,1,0,POINT (-122.29061 47.70829)
105446,77221401,Signal,16959,158794036,Matthews Beach,2022-09-01,2023-04-26 00:26:57.504,NaN,False,2,0,0,POINT (-122.29082 47.70483)


In [47]:
#save labels to a new json file
labels.to_file("attributes_filtered.json", driver="GeoJSON")